In [23]:
import json
import requests
import praw
import numpy
import time
import pickle
import pandas as pd


# Not needed
"""
reddit = praw.Reddit(client_id='your_client_id', 
                     client_secret='your_client_secret', 
                     user_agent='your_user_agent', 
                     username= keys.username , 
                     password= keys.password)
"""

# STATIC VARIABLES
base_url = 'https://api.pushshift.io/reddit/search/comment/'
wanted_count_auth = 20
wanted_count_sub = 20
unwanted_users = ["[deleted]", "AutoModerator"]
unwanted_count = len(unwanted_users)
request_limit = 120
request_count = 0
request_sum = 0
sleep_time = 60


    
# FUNCTIONS
def getKeysFromDict(data):
    users = []
    for obj in data:
        if obj['key'] not in unwanted_users:
            users.append(obj['key'])
    return users

def getMostActiveAuthorsInSubReddit(subreddit, log):
    global request_count
    global request_sum
    
    time.sleep(1)
    retry_count = 0
    retry_limit = 10

    if request_count >= request_limit:
        print("\r","Sleeping...", end = "")
        log.write("Sleeping... \n")
        time.sleep(sleep_time)
        request_count = 0
        print("\r","Reseting request_count to 0", end = "")
        log.write("Reseting request_count to 0 \n")
        print("\r","Resuming...", end = "")
        log.write("Resuming... \n")
        
    url = base_url + '?since=100d&subreddit={}&aggs=author&agg_size={}'.format(subreddit, wanted_count_auth + unwanted_count)
    response_raw = requests.get(url)
    request_count += 1
    request_sum += 1
    

    #print("\r","Requests performed:\t\t\t", request_count, end = "")
    #print("\r","Total requests performed:\t\t", request_sum, end = "")
    #print("Percent completed:\t\t\t", 555, end = "\r")
    
    log.write("Requests performed:\t\t\t {} \n".format(request_count))
    log.write("Total requests performed:\t\t {} \n".format(request_sum))
    #log.write("Percent completed:\t\t\t {} \n".format(555))
    
    while response_raw.status_code != 200 or retry_count == retry_limit:
        print("\r", "Received {} error!".format(response_raw.status_code), end = "")
        log.write("Received {} error! \n".format(response_raw.status_code))
        print("\r","Sleeping...", end = "")
        log.write("Sleeping... \n")
        time.sleep(sleep_time)
        print("\r","Reseting request_count...", end = "")
        log.write("Reseting request_count... \n")
        request_count = 0
        print("\r","Performing request", end = "")
        log.write("Performing request \n")
        response_raw = requests.get(url)
        print("\r",url, response_raw, end = "")
        log.write("{} {} \n".format(url, response_raw))
        response = response_raw.json()
        request_count += 1
        request_sum += 1
        retry_count +=1

    #print("\r",url, response_raw, end = "")
    response = response_raw.json()

        
        
    
    #print(response)
    fetched_authors = getKeysFromDict(response['aggs']['author'])
    #print("\r","*******************", end = "")
    #print("\r",end = "")
    log.write("******************* \n")
    log.write("\n")

    return fetched_authors


def getMostActiveSubRedditForAuthor(author, log):
    global request_count
    global request_sum
    
    time.sleep(1)
    retry_count = 0
    retry_limit = 10
    
    if request_count >= request_limit:
        print("\r","Sleeping...", end = "")
        log.write("Sleeping... \n")
        time.sleep(sleep_time)
        request_count = 0
        print("\r","Reseting request_count to 0", end = "")
        print("\r","Resuming...", end = "")
        
        log.write("Reseting request_count to 0 \n")
        log.write("Resuming... \n")
        
    url = base_url + '?since=100d&author={}&aggs=subreddit&agg_size={}'.format(author, wanted_count_sub)
    response_raw = requests.get(url)
    
    request_count += 1
    request_sum += 1

    #print("\r","Requests performed:\t\t\t", request_count, end = "")
    #print("\r","Total requests performed:\t\t", request_sum, end = "")
    #print("\r","Percent completed:\t\t\t", 555, end = "")   
    
    log.write("Requests performed:\t\t\t {} \n".format(request_count))
    log.write("Total requests performed:\t\t {} \n".format(request_sum))
    #log.write("Percent completed:\t\t\t {} \n".format(555)) 
    
    while response_raw.status_code != 200 or retry_count == retry_limit:
        print("\r","Received {} error!".format(response_raw.status_code), end = "")
        log.write("Received {} error! \n".format(response_raw.status_code))
        print("\r","Sleeping...", end = "")
        log.write("Sleeping... \n")
        time.sleep(sleep_time)
        print("\r","Reseting request_count...", end = "")
        log.write("Reseting request_count... \n")
        request_count = 0
        print("\r","Performing request", end = "")
        log.write("Performing request \n")
        response_raw = requests.get(url)
        print("\r",url, response_raw, end = "")
        log.write("{} {} \n".format(url, response_raw))
        response = response_raw.json()
        request_count += 1
        request_sum += 1
        retry_count +=1

        
        
    # print response
    #print("\r",url, response_raw, end = "")
    response = response_raw.json()
    
    fetched_subreddits = getKeysFromDict(response['aggs']['subreddit'])
    #print("Detected subreddits:\t\t\t", fetched_subreddits)
    log.write("Detected subreddits:\t\t\t \n".format(fetched_subreddits))
    for sub in used_subreddits:
            if (sub not in used_subreddits):
                used_subreddits.append(sub)
    #print("\r","*******************")
    #print("\r")
    log.write("******************* \n")
    log.write("\n")
    
    time.sleep(1)
    return fetched_subreddits

In [24]:
%%time

# Global variables
used_subreddits = []
used_authors = []
start = time.time()
log = open('log.txt', 'w+')


def get_relations(relational_array, subreddit, wanted_subcount):
    global request_count
    global request_sum
    global start
    time_elapsed = time.time() - start
    #print("\r, ""Time elapsed: \t\t\t\t", round(time_elapsed,2), "seconds", end = "")
    log.write("Time elapsed: \t\t\t\t {} seconds \n".format(round(time_elapsed,2)))

    
    if wanted_subcount == 0:
        return []
    
    # Add subreddit to used_subreddits 
    used_subreddits.append(subreddit)
    
    # Retrieve users from subreddit and add to request_count
    users = getMostActiveAuthorsInSubReddit(subreddit, log)

    # Retrieve subs from active users in subreddit
    for user in users:
            subs = getMostActiveSubRedditForAuthor(user, log)
            used_authors.append(user)

            # Enter new level
            for sub in subs:
                if sub != subreddit:
                    relational_array.append([subreddit, sub])

                if sub not in used_subreddits:
                    relational_array.extend(get_relations([], sub, wanted_subcount - 1))
    
    # Dump data to file
    pickle.dump(relational_array, open("relational_array.dat", "wb"))
    
    return relational_array


rels = get_relations([], 'collapse', 2)
rels
log.write("################################################################################\n")
log.write("################################################################################\n")
log.write("################################################################################\n")
log.write("SESSION ENDED\n")
log.write("################################################################################\n")
log.write("################################################################################\n")
log.write("################################################################################\n")
log.close()

 Resuming...quest_count to 0ddit/search/comment/?since=100d&author=RespectableBloke69&aggs=subreddit&agg_size=20 <Response [200]>

ConnectionError: ('Connection aborted.', OSError("(10053, 'WSAECONNABORTED')"))

In [22]:
rel_array = pickle.load(open("relational_array.dat", "rb"))
rel_array

[['TheFacebookDelusion', 'facepalm'],
 ['TheFacebookDelusion', 'badwomensanatomy'],
 ['TheFacebookDelusion', 'WordsWithFriends'],
 ['TheFacebookDelusion', 'FanTheories'],
 ['TheFacebookDelusion', 'CampCult'],
 ['TheFacebookDelusion', 'DebateReligion'],
 ['TheFacebookDelusion', 'silentcinema'],
 ['TheFacebookDelusion', 'Teachers'],
 ['TheFacebookDelusion', 'AskAnAmerican'],
 ['TheFacebookDelusion', 'AskReddit'],
 ['TheFacebookDelusion', 'ChoosingBeggars'],
 ['TheFacebookDelusion', 'Ohio'],
 ['TheFacebookDelusion', 'teaching'],
 ['TheFacebookDelusion', 'education'],
 ['TheFacebookDelusion', 'atheism'],
 ['TheFacebookDelusion', 'pics'],
 ['TheFacebookDelusion', 'TrueCrimePodcasts'],
 ['TheFacebookDelusion', 'AmItheAsshole'],
 ['TheFacebookDelusion', 'dreamcatcher'],
 ['TheFacebookDelusion', 'uwaterloo'],
 ['TheFacebookDelusion', 'waterloo'],
 ['TheFacebookDelusion', 'buildapc'],
 ['TheFacebookDelusion', 'HongKong'],
 ['TheFacebookDelusion', 'AskReddit'],
 ['TheFacebookDelusion', 'LaCasaDe

In [46]:
df = pd.DataFrame(rel_array, columns = ["from", "to"])

In [48]:
df.to_csv("data.csv", index=False)

### 